In [65]:
import pulp
from pulp import *
import pandas as pd
import numpy as np

In this optimization problem we have to optimize the cost of peanuts paste having the ingredients, proportions required, number of pastes to produce, cost and availability of each ingredient.

In [144]:
df = pd.DataFrame(data = {'Ingredients' : ['peanuts', 'sunflower seeds', 'honey'], 'Cost(rub/kg)' : [120, 60, 250], 'Availability' : [30, 50, 10]})

In [145]:
df

,Ingredients,Cost(rub/kg),Availability
0,peanuts,120,30
1,sunflower seeds,60,50
2,honey,250,10


We need to make 2 types of paste: economy and premium. Economy (>40% of peanuts) and Premium (>60% of peanuts). Each can is 250 grams (0.25 kg)

Sunflower seeds can be used at most 25%. 

Our demand is to produce 100 economy and 100 premium pastes. 

In [146]:
# Initiate our problem 
model = pulp.LpProblem('Cost minimising blending problem', pulp.LpMinimize)

In [147]:
model

Cost minimising blending problem:
MINIMIZE
None
VARIABLES

In [148]:
paste_types = ['economy', 'premium']
ingredients = ['peanuts', 'sunflower seeds', 'honey']

In [149]:
ing_weight = pulp.LpVariable.dicts('weight kg', ((i, j) for i in paste_types for j in ingredients), lowBound = 0, cat = 'Continuous')

In [150]:
ing_weight

{('economy', 'peanuts'): weight_kg_('economy',_'peanuts'),
 ('economy', 'sunflower seeds'): weight_kg_('economy',_'sunflower_seeds'),
 ('economy', 'honey'): weight_kg_('economy',_'honey'),
 ('premium', 'peanuts'): weight_kg_('premium',_'peanuts'),
 ('premium', 'sunflower seeds'): weight_kg_('premium',_'sunflower_seeds'),
 ('premium', 'honey'): weight_kg_('premium',_'honey')}

In [151]:
model += (pulp.lpSum([120 * ing_weight[(i, 'peanuts')] + 60 * ing_weight[(i, 'sunflower seeds')] + 250 * ing_weight[(i, 'honey')] for i in paste_types]))

In [152]:
model

Cost minimising blending problem:
MINIMIZE
250*weight_kg_('economy',_'honey') + 120*weight_kg_('economy',_'peanuts') + 60*weight_kg_('economy',_'sunflower_seeds') + 250*weight_kg_('premium',_'honey') + 120*weight_kg_('premium',_'peanuts') + 60*weight_kg_('premium',_'sunflower_seeds') + 0
VARIABLES
weight_kg_('economy',_'honey') Continuous
weight_kg_('economy',_'peanuts') Continuous
weight_kg_('economy',_'sunflower_seeds') Continuous
weight_kg_('premium',_'honey') Continuous
weight_kg_('premium',_'peanuts') Continuous
weight_kg_('premium',_'sunflower_seeds') Continuous

In [153]:
# constraints
# 100 economy and 100 premium pastes at 0.25 kg

model += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == 100 * 0.25
model += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == 100 * 0.25

# economy paste has >= 40% peanuts, premium >= 60%
model += ing_weight['economy', 'peanuts'] >= (0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))
model += ing_weight['premium', 'peanuts'] >= (0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# economy paste must be <= 25% sunflower seeds
model += ing_weight['economy', 'sunflower seeds'] <= (0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))
model += ing_weight['premium', 'sunflower seeds'] <= (0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# we have 30 kg of peanuts, 50 kg of sunflower seeds and 10 kg of honey
model += pulp.lpSum([ing_weight[i, 'peanuts'] for i in paste_types]) <= 30
model += pulp.lpSum([ing_weight[i, 'sunflower seeds'] for i in paste_types]) <= 50
model += pulp.lpSum([ing_weight[i, 'honey'] for i in paste_types]) <= 10



In [154]:
model

Cost minimising blending problem:
MINIMIZE
250*weight_kg_('economy',_'honey') + 120*weight_kg_('economy',_'peanuts') + 60*weight_kg_('economy',_'sunflower_seeds') + 250*weight_kg_('premium',_'honey') + 120*weight_kg_('premium',_'peanuts') + 60*weight_kg_('premium',_'sunflower_seeds') + 0
SUBJECT TO
_C1: weight_kg_('economy',_'honey') + weight_kg_('economy',_'peanuts')
 + weight_kg_('economy',_'sunflower_seeds') = 25

_C2: weight_kg_('premium',_'honey') + weight_kg_('premium',_'peanuts')
 + weight_kg_('premium',_'sunflower_seeds') = 25

_C3: - 0.4 weight_kg_('economy',_'honey')
 + 0.6 weight_kg_('economy',_'peanuts')
 - 0.4 weight_kg_('economy',_'sunflower_seeds') >= 0

_C4: - 0.6 weight_kg_('premium',_'honey')
 + 0.4 weight_kg_('premium',_'peanuts')
 - 0.6 weight_kg_('premium',_'sunflower_seeds') >= 0

_C5: - 0.25 weight_kg_('economy',_'honey')
 - 0.25 weight_kg_('economy',_'peanuts')
 + 0.75 weight_kg_('economy',_'sunflower_seeds') <= 0

_C6: - 0.25 weight_kg_('premium',_'honey')
 - 0

In [155]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [156]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print('The weight of {0} in {1} pastes is {2} kg'.format(var[1], var[0], var_value))

The weight of peanuts in economy pastes is 11.25 kg
The weight of sunflower seeds in economy pastes is 6.25 kg
The weight of honey in economy pastes is 7.5 kg
The weight of peanuts in premium pastes is 18.75 kg
The weight of sunflower seeds in premium pastes is 6.25 kg
The weight of honey in premium pastes is 0.0 kg


In [157]:
total_cost = pulp.value(model.objective)
print('The total cost is Russian Rubles {} for 100 economy pastes and 100 premium pastes'.format(round(total_cost, 3)))

The total cost is Russian Rubles 6225.0 for 100 economy pastes and 100 premium pastes
